In [ ]:
import torch
from torch import nn
from utils.loader import load_data_fashion_mnist
from utils.train import train_ch3,predict_ch3

batch_size = 256
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'使用设备类型: {device}')
train_iter, test_iter = load_data_fashion_mnist(batch_size)

num_inputs, num_outputs, num_hiddens = 784, 10, 256

# by hand
W1 = nn.Parameter(torch.randn(num_inputs, num_hiddens, requires_grad=True,device=device) * 0.01)
b1 = nn.Parameter(torch.zeros(num_hiddens, requires_grad=True,device=device))
W2 = nn.Parameter(torch.randn(num_hiddens, num_outputs, requires_grad=True,device=device) * 0.01)
b2 = nn.Parameter(torch.zeros(num_outputs, requires_grad=True,device=device))

params = [W1, b1, W2, b2]

def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

def net(X):
    X = X.reshape((-1, num_inputs))
    H = relu(X@W1 + b1)  # 这里“@”代表矩阵乘法
    return (H@W2 + b2)
loss = nn.CrossEntropyLoss(reduction='none')
num_epochs, lr = 10, 0.1
updater = torch.optim.SGD(params, lr=lr)
train_ch3(net, train_iter, test_iter, loss, num_epochs, updater)

In [ ]:
from utils.train import predict_ch3

predict_ch3(net, test_iter)

In [ ]:
# by nn
from torch import nn

net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256,device=device),
                    nn.ReLU(),
                    nn.Linear(256, 10,device=device))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)
batch_size, lr, num_epochs = 128, 0.1, 20
loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=lr)

train_iter, test_iter = load_data_fashion_mnist(batch_size)
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)